# Applied Data Analysis Project
**Team**: ToeStewBrr - Alexander Sternfeld, Marguerite Thery, Antoine Bonnet, Hugo Bordereaux

**Dataset**: CMU Movie Summary Corpus


## CoreNLP Analysis

[**CoreNLP**](https://nlp.stanford.edu/software/) is an incredible natural language processing toolkit created at Stanford University. CoreNLP is applied through a **pipeline** of sequential analysis steps called annotators. The full list of available annotators is available [here](https://stanfordnlp.github.io/CoreNLP/annotators.html). 

As described by its creators: 

*"CoreNLP is your one stop shop for natural language processing in Java! CoreNLP enables users to derive linguistic annotations for text, including token and sentence boundaries, parts of speech, named entities, numeric and time values, dependency and constituency parses, coreference, sentiment, quote attributions, and relations. CoreNLP currently supports 8 languages: Arabic, Chinese, English, French, German, Hungarian, Italian, and Spanish."* 

You can create your own pipeline to extract the desired information. You can try it out for yourself in this [online shell](https://corenlp.run).

### Loading data
We first load data files and download the pre-processed dataframes. 

In [1]:
from zipfile import ZipFile
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np

from load_data import *
from coreNLP_analysis import *
from extraction import *

download_data(coreNLP=False)
plot_df = load_plot_df()
movie_df = load_movie_df()
char_df = load_char_df()
names_df = load_names_df()
cluster_df = load_cluster_df()

### 1. Exploring pre-processed CoreNLP data

The authors of the Movie CMU dataset used CoreNLP to parse each plot summary to extract various linguistic insights. In this section, we explore how much information we can gather from these pre-processed files. 

We will use *Harry Potter*'s character throughout this section.

#### 1.1. Character data

For any character, we first extract related information from the provided name clusters and character metadata.

In [6]:
# Given character, extract all pre-processed dataframe data
char_name = 'Harry Potter'
movie_ids = list(char_df[char_df['Character name'] == char_name]['Wikipedia ID'])

print('Movies with character', char_name, ':')
print('\tMovie IDs:', movie_ids)

movie_id = movie_ids[3]
movie_name = movie_df.loc[movie_df['Wikipedia ID'] == movie_id]['Name'].iloc[0]

print('Selecting as example: \n\tMovie ID:', movie_id, '\n\tMovie title:', movie_name)


Movies with character Harry Potter :
	Movie IDs: [858575, 667372, 670407, 31941988, 9834441, 667368, 667371, 667361, 667361]
Selecting as example: 
	Movie ID: 31941988 
	Movie title: Harry Potter and the Deathly Hallows – Part 2


#### 1.2. Extracting sentences

We now extract information from the CoreNLP plot summary analysis. The authors of the dataset stored the analysis output of each movie into a `.xml` file. Each file has a tree structure detailing each word of each sentence as well as the parsed sentence in tree form. 

We now extract all parsed sentences from the `.xml` files. 

A **parsed sentence** is a syntactic analysis tree, where each word is a leaf tagged by its lexical function (e.g. *VBZ* for verbs or *DT* for determinants). Semantic interactions between different words are also indicated within the structure of the tree. 

In [7]:
# Extract the tree of xml file and all parsed sentences
tree = get_tree(movie_id)
sentences = get_parsed_sentences(tree)

# Picking the fifth sentence as example
parsed_str = sentences[5]
print(parsed_str)
print_tree(parsed_str)

(ROOT (S (PP (IN In) (NP (NP (NNP Bellatrix) (POS 's)) (NN vault))) (, ,) (NP (NNP Harry)) (VP (VBZ discovers) (SBAR (S (NP (DT the) (NNP Horcrux)) (VP (VBZ is) (NP (NP (NNP Helga) (NNP Hufflepuff) (POS 's)) (NN cup)))))) (. .))) 
                                                ROOT                                                 
                                                 |                                                    
                                                 S                                                   
                _________________________________|_________________________________________________   
               |             |    |                               VP                               | 
               |             |    |        _______________________|____                            |  
               |             |    |       |                           SBAR                         | 
               |             |    |       |         

#### 1.3. Extracting characters

We also want to extract all character names directly from the xml file. Note that we aggregate consecutive words tagged as NNP (noun, proper, singular) as the same character name (this assumes that plot summaries never contain two distinct names side by side without delimiting punctuation). This is a reasonable assumption since list of names are almost always separated by commas. 

In [8]:
characters = get_characters(tree)
print(characters[:20])

['Voldemort', 'Albus Dumbledore', 'Severus Snape', 'Dobby', 'Harry Potter', 'Ron', 'Hermione', 'Griphook', 'Harry', 'Ollivander', 'Ollivander', 'Draco Malfoy', 'Malfoy', 'Harry', 'Harry', 'Helga Hufflepuff', 'Griphook', 'Harry', 'Voldemort', 'Griphook']


Notice that some characters are sometimes mentioned by their full name, and sometimes by a partial name (e.g. Harry Potter is most often mentioned as simply Harry). To get a more precise idea of how many times each character is mentioned, we wish to denote each character by their full name, i.e. the longest version of their name that appears in the plot summary. 

*NOTE*: The dataset has the character metadata of only a third of the movies, so we need to extract full names from the plot summary itself and not the provided dataframes. 

To optimize full name lookup, for each plot summary we construct a dictionary which stores as key every partial name mentioned, and as corresponding values the full name of each character.  

In [9]:
char_name = 'Albus'
full_name = get_full_name(char_name, characters)
print('Example: the full name of "{}" is "{}".'.format(char_name,full_name))
print('Full name dictionary:', full_name_dict(characters))

Example: the full name of "Albus" is "Albus Dumbledore".
Full name dictionary: {'Voldemort': 'Voldemort', 'Albus Dumbledore': 'Albus Dumbledore', 'Severus Snape': 'Severus Snape', 'Dobby': 'Dobby', 'Harry Potter': 'Harry Potter', 'Ron': 'Ron', 'Hermione': 'Hermione Weasley', 'Griphook': 'Griphook', 'Harry': 'Harry Potter', 'Ollivander': 'Ollivander', 'Draco Malfoy': 'Draco Malfoy', 'Malfoy': 'Draco Malfoy', 'Helga Hufflepuff': 'Helga Hufflepuff', 'Rowena Ravenclaw': 'Rowena Ravenclaw', 'Hogsmeade': 'Hogsmeade', 'Aberforth Dumbledore': 'Aberforth Dumbledore', 'Ariana': 'Ariana', 'Neville Longbottom': 'Neville Longbottom', 'Snape': 'Severus Snape', 'Minerva McGonagall': 'Minerva McGonagall', 'Luna Lovegood': 'Luna Lovegood', 'Helena Ravenclaw': 'Helena Ravenclaw', 'Gregory Goyle': 'Gregory Goyle', 'Blaise Zabini': 'Blaise Zabini', 'Nagini': 'Nagini', 'Fred': 'Fred', 'Lily': 'Lily', 'James': 'James', 'Dumbledore': 'Albus Dumbledore', 'Neville': 'Neville Longbottom', 'Molly Weasley': 'Moll

We can now extract the most mentioned characters in any plot summary, in descending order of frequency. We can then see that Harry Potter is indeed the main character of the movie, as he is mentioned 26 times, more than any other character in the summary.  

In [10]:
char_mentions = most_mentioned(movie_id)
print(char_mentions)

[('Harry Potter', 26), ('Voldemort', 21), ('Severus Snape', 11), ('Ron', 6), ('Hermione Weasley', 6), ('Albus Dumbledore', 5), ('Griphook', 3), ('Draco Malfoy', 3), ('Neville Longbottom', 3), ('Nagini', 3), ('Ollivander', 2), ('Lily', 2), ('Dobby', 1), ('Helga Hufflepuff', 1), ('Rowena Ravenclaw', 1), ('Hogsmeade', 1), ('Aberforth Dumbledore', 1), ('Ariana', 1), ('Minerva McGonagall', 1), ('Luna Lovegood', 1), ('Helena Ravenclaw', 1), ('Gregory Goyle', 1), ('Blaise Zabini', 1), ('Fred', 1), ('James', 1), ('Molly Weasley', 1), ('Ginny Potter', 1)]


 #### 1.4. Extracting interactions

We are also interested in character interactions. We can use the number of common mentions of two characters in the same sentence as a proxy for the number of interactions. For any movie, we find the number of common mentions (i.e. interactions) for each pair of characters. 

In [11]:
char_pairs = character_pairs(movie_id, plot_df)
print(char_pairs[:10])

[(('Hermione Weasley', 'Ron'), 4), (('Harry Potter', 'Voldemort'), 4), (('Albus Dumbledore', 'Voldemort'), 3), (('Albus Dumbledore', 'Severus Snape'), 2), (('Harry Potter', 'Hermione Weasley'), 2), (('Harry Potter', 'Ron'), 2), (('Nagini', 'Voldemort'), 2), (('Harry Potter', 'Lily'), 2), (('Albus Dumbledore', 'Harry Potter'), 2), (('Severus Snape', 'Voldemort'), 1)]


In [12]:
main_interaction = character_pairs(movie_id, plot_df)[0][0]
print('Main interaction in the movie:', main_interaction)

Main interaction in the movie: ('Hermione Weasley', 'Ron')


#### 1.5. Extracting characters and interactions of all movies

We will now use the above code to obtain the main character and main interaction for every plot summary. 

*NOTE*: This code takes a while to run, so you can load the analysis from a pre-processed file instead.  

In [13]:
# NOTE: If we've already run this code, we can load the dataframe from a file
plot_char_filename = 'Data/MovieSummaries/plot_characters.csv'
pairs_df = pd.read_csv(plot_char_filename, sep='\t', index_col=0)
pairs_df

,Wikipedia ID,Summary,Main character,Number of mentions,Main interaction,Number of interactions
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...",Shlykov,1.0,"('Lyosha', 'Shlykov')",1.0
1,31186339,The nation of Panem consists of a wealthy Capi...,Katniss,18.0,"('Katniss', 'Peeta Mellark')",2.0
2,20663735,Poovalli Induchoodan is sentenced for six yea...,Maranchery Karunakara Menon,9.0,"('Manapally Madhavan Nambiar', 'judge Menon')",1.0
3,2231378,"The Lemon Drop Kid , a New York City swindler,...",Charley,18.0,NaN,NaN
4,595909,Seventh-day Adventist Church pastor Michael Ch...,Lindy,7.0,"('Azaria', 'Lindy')",1.0
...,...,...,...,...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho...",Reema,1.0,"('Muslim', 'Reema')",1.0
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look...",Leo Andreyev,7.0,NaN,NaN
42300,35102018,American Luthier focuses on Randy Parsons’ tra...,Randy Parsons,4.0,NaN,NaN
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se...",Abdur Rehman Khan,9.0,"('Abdur Rehman Khan', 'Amina')",1.0


In [14]:
# Otherwise: get main character and number of mentions for each movie and store it into a file (takes a while to run)
if not os.path.exists(plot_char_filename):
    pairs_df = plot_df.copy(deep=True)
    pairs_df['Main character'] = pairs_df['Wikipedia ID'].apply(most_mentioned)
    pairs_df['Number of mentions'] = pairs_df['Main character'].apply(lambda x: np.nan if x is None else x[0][1])
    pairs_df['Main character'] = pairs_df['Main character'].apply(lambda x: np.nan if x is None else x[0][0])

    # Get main pairs of characters for each movie and number of interactions 
    pairs_df['Main interaction'] = pairs_df['Wikipedia ID'].apply(lambda x: character_pairs(x, plot_df))
    pairs_df['Number of interactions'] = pairs_df['Main interaction'].apply(lambda x: np.nan if x is None else x[0][1])
    pairs_df['Main interaction'] = pairs_df['Main interaction'].apply(lambda x: np.nan if x is None else x[0][0])

    # Store data into csv file
    pairs_df.to_csv(plot_char_filename, sep='\t')
    pairs_df

In conclusion, the coreNLP files provided with the datasets are useful to extract the characters mentioned. 

 However, our goal is to extract love relationships as well as the persona of characters in love. Using common mentions as a proxy for love relationships is a vulgar approximation and so we must run our own NLP analysis on the plot summaries to extract useful information. 

### 2. Custom CoreNLP Analysis

We now use a **custom CoreNLP pipeline** to analyze the plot summaries. For now, due to the weakness of our available computing power, we only analyze romantic comedy movies. 


#### 2.1. Data preparation

We extract the romantic comedy plot summaries that we will pass through our pipeline and store them as `.txt` files to be able to run them through the new coreNLP pipeline. 

In [15]:
# For later use: romance_genres = ['Romantic comedy', 'Romance Film', 'Romantic drama', 'Romantic fantasy', 'Romantic thriller']

# Get a dataframe with romantic movies and their corresponding plots
romance_genres = ['Romantic comedy'] 
rom_com_plots = get_plots(romance_genres, movie_df, plot_df)
#display(rom_com_plots)

# Store each plot summary as .txt file
for index, row in rom_com_plots.iterrows():
    # If directory doesn't exist, create it
    if not os.path.exists('Data/MovieSummaries/RomancePlots'):
        os.makedirs('Data/MovieSummaries/RomancePlots')
    with open("Data/MovieSummaries/RomancePlots/{}.txt".format(row['Wikipedia ID']), 'w', encoding='utf8') as f:
        if type(row['Summary']) == str:
            f.write(row['Summary'])
            f.close()

#### 2.1. Custom CoreNLP pipeline

Our custom pipeline consists of the following annotators: 

1. [Tokenization (tokenize)](https://stanfordnlp.github.io/CoreNLP/tokenize.html): Turns the whole text into tokens. 

2. [Parts Of Speech (POS)](https://stanfordnlp.github.io/CoreNLP/pos.html): Tags each token with part of speech labels (e.g. determinants, verbs and nouns). 

3. [Lemmatization (lemma)](https://stanfordnlp.github.io/CoreNLP/lemma.html): Reduces each word to its lemma (e.g. *was* becomes *be*). 

4. [Named Entity Recognition (NER)](https://stanfordnlp.github.io/CoreNLP/ner.html): Identifies named entities from the text, including characters, locations and organizations. 

5. [Constituency parsing (parse)](https://stanfordnlp.github.io/CoreNLP/parse.html): Performs a syntactic analysis of each sentence in the form of a tree. 

6. [Coreference resolution (coref)](https://stanfordnlp.github.io/CoreNLP/coref.html): Aggregates mentions of the same entities in a text (e.g. when 'Harry' and 'he' refer to the same person). 

7. [Dependency parsing (depparse)](https://stanfordnlp.github.io/CoreNLP/depparse.html): Syntactic dependency parser. 

8. [Natural Logic (natlog)](https://stanfordnlp.github.io/CoreNLP/natlog.html): Identifies quantifier scope and token polarity. Required as preliminary for OpenIE. 

9. [Open Information Extraction (OpenIE)](https://stanfordnlp.github.io/CoreNLP/openie.html): Identifies relation between words as triples *(subject, relation, object of relation)*. We use this to extract relationships between characters, as well as character traits. 

10. [Knowledge Base Population (KBP)](https://stanfordnlp.github.io/CoreNLP/kbp.html): Identifies meaningful relation triples. 


#### 2.2. Running our pipeline

We now run our own CoreNLP analysis on the plot summaries. This allows us to extract love relationships from the plot summaries much more accurately.

**Goal**: Run our custom CoreNLP pipeline. 

**Recommendation**: Be careful about memory storage (takes a lot of memory to run!)

**Prerequisite**: [java](https://www.java.com). 

**Installation steps**:
1. Download the CoreNLP toolkit [here](https://stanfordnlp.github.io/CoreNLP/download.html).

2. Data preparation: Extract plot summaries for romantic comedies into `.txt` files. Create a filelist containing the name of all the files which need to be processed using the following command: 

        find RomancePlots/*.txt > filelist.txt

3. Change directory (`cd`) into the downloaded `stanford-corenlp` directory. 
        
4. Run the custom CoreNLP pipeline via your terminal using the following command:

        java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,pos,lemma,ner,parse,coref,depparse,natlog,openie,kbp -coref.md.type RULE -filelist filelist.txt -outputDirectory RomancePlotsOutputs/ -outputFormat xml

The analysis outputs are now stored as `.xml` files in the `RomancePlotsOutputs` directory. We now unzip them. RomancePlotsOutputs has 1491 readable files. 

In [16]:
# Extract all the romance plots xml files
# If r`RomancePlotsOutputs directory doesn't exist, create it
if not os.path.exists('Data/CoreNLP/RomancePlotsOutputs'):
   with ZipFile('CoreNLP/RomanceOutputs.zip', 'r') as zipObj:
      zipObj.extractall('')

### 3. Extracting data

Now that we have run the coreNLP pipeline and that the analysis of each movie has been a stored into a .xml output file, we can extract the information from these files. 

We will first extract the attributes and actions related to entities in the plot summaries. We will extract verbs and attributes independently. 
Agent verb: character does the action
Patient verb: character is the object of the action
Attributes: character attributes

**Dependency parsing extraction**
| Relation | Description |  Type  |  Example |
|---|---|---|---|
| obl:agent | Agent | Agent verb | 'They were rescued by Dumbledore' -> obl:agent(rescued, Dumbledore) |
| nsubj  | Nominal subject | Agent verb | 'Harry confronts Snape' -> nsubj(confronts, Harry) |
| nsubj:pass | Passive nominal subject | Patient verb | 'Goyle casts a curse and is burned to death' -> nsubj:pass(burned, Goyle)|
| nsubj:xsubj | Indirect nominal subject | Patient verb | 'Goyle casts a curse and is unable to control it' -> nsubj:xsubj(control, Goyle)|
| obj |  Direct object | Patient verb | 'To protect Harry' -> obj(protect, Harry) |
| appos | Appositional modifier | Attribute | 'Harry's mother, Lily' -> appos(mother, Lily) |
| amod | Adjectival modifier | Attribute | 'After burrying Dobby' -> amod(Dobby, burrying) |
| nmod:poss | Possessive nominal modifier | Attribute | 'Snape's memories' -> nmod:poss(memories, Snape) |
| nmod:of | 'Of' nominal modifier | Attribute |'With the help of Griphook' -> nmod:of(help, Griphook) |

We will also extract KBP outputs, which stores data including the main role, spouse, age and religion for each character if specified. 

**KBP Extraction**
| Attributes | Relation name | 
|---|---|
| Main role | per:title |
| Marital relationship | per:spouse  |  
| Age  | per:age | 
| Religion  | per:religion | 

[KBP documentation](https://stanfordnlp.github.io/CoreNLP/kbp.html)

In [2]:
descriptions, relations = extract_descriptions_relations(log_interval=1000)

Extracting character descriptions & relations...
Progress: 0/1000 (0.0%)


In [9]:
# Get all non Nan values for death
death = descriptions[descriptions['death'].notna()]
# how many rows
print('Number of rows:', len(death))
death

Number of rows: 15


,movie_id,character,agent_verbs,patient_verbs,attributes,title,religion,age,death,children
147,318772,Jonathan Zimmermann,"[drives, contacted, knowing, refuses, asks, dr...","[shake, visits, confronts, convinced, made, te...","[offer, involved, upsets, framer, wife, gettin...",picture framer,NaN,NaN,disease,NaN
263,5017604,Johnson,[dies],NaN,[Bumpy],NaN,NaN,NaN,heart attack,NaN
315,31232513,Sam,"[put, introduces, decides, broods, tries, foll...","[united, accepts, finds, leaving, rejected, pl...","[boss, expense, shot, home, barterer, young]",NaN,NaN,NaN,cancer,NaN
316,31232513,Hannah,"[finds, tells, insists]",[disgusts],[girlfriend],NaN,NaN,NaN,cancer,NaN
317,31232513,Jerry,"[united, rejected, visit, choose]",NaN,"[kit, money, daughter, house, father]",record - producer,NaN,NaN,cancer,NaN
422,3237184,Pat Chambers,[shows],NaN,NaN,Capt.,NaN,NaN,hit - and - run car accident,NaN
423,3237184,Pat Chambers,[shows],NaN,NaN,cleaner,NaN,NaN,hit - and - run car accident,NaN
932,19992131,Anna Taylor,"[accepts, asks, unsuccessful, clings, tells, m...","[escape, shown, knew, sees, injecting, reveale...",[grave],teacher,NaN,NaN,suffocation,NaN
1976,3917900,Lydell Hammond,NaN,"[assassinate, named]","[funeral, Sr., Jr., opponent]",NaN,NaN,NaN,accident,NaN
1996,5566154,David Sheldon,NaN,[defends],"[owner, partner]",owner,NaN,NaN,disease,NaN


As an example, we extract the description of each character in the Harry Potter movie, which is composed of all agent verbs, patient verbs and attributes present in the plot summary. We also extract the love relationships in the movie, if present. 

In [30]:
example_filename = f'Data/CoreNLP/PlotsOutputs/667372.xml'

tree = ET.parse(example_filename)
descriptions_df, relations_df = get_descriptions_relations(tree)
display(descriptions_df)
display(relations_df)

,movie_id,character,agent_verbs,patient_verbs,attributes,title,religion,age
0,667372,Albus Dumbledore,"[vouches, tells, exhorts, introduces, arrive, ...","[want, pull, tells]","[office, pensieve, memories]",headmaster,NaN,NaN
1,667372,Harry Potter,"[enters, suspicious, found, have, tells, finds...","[provides, compete, drives, arrive, take, tell...","[parents, forces, helping, name, curiosity, de...",NaN,NaN,NaN
2,667372,Voldemort,"[responsible, waiting, disgorge, returning, to...","[rejuvenates, meaning]","[followers, returned, forces]",NaN,NaN,NaN
3,667372,Ron,"[becomes, have]","[drives, free]",[sister],NaN,NaN,NaN
4,667372,Severus Snape,[arrive],NaN,[integrity],NaN,NaN,NaN
5,667372,Viktor Krum,"[attempts, incapacitates]","[do, cursed]","[attacking, Hogwarts]",NaN,NaN,NaN
6,667372,Cedric Diggory,"[provides, performs]","[stops, trapped, put]","[death, parents, assisted, body]",student,NaN,NaN
7,667372,Hermione,[attending],NaN,[sister],NaN,NaN,NaN
8,667372,Wormtail,[waiting],NaN,NaN,NaN,NaN,NaN
9,667372,Fleur Delacour,NaN,[incapacitates],[sister],NaN,NaN,NaN


,subject,object,movie_id


We can now extract the character descriptions of all characters in all non-romantic movies and store the results in a dataframe. 

In [15]:
description_path = 'Data/CoreNLP/descriptions.csv'
relations_path = 'Data/CoreNLP/relations.csv'

if not os.path.exists(description_path) and not os.path.exists(relations_path):

    # Extract descriptions and relations from all xml files
    output_dir = 'Data/CoreNLP/PlotsOutputs'
    descriptions, relations = extract_descriptions_relations(output_dir)

    # Save descriptions and relations into csv files
    descriptions.to_csv(description_path, sep='\t')
    relations.to_csv(relations_path, sep='\t')

# If we've already run the extraction, we can load the dataframe from a file
else: 
    descriptions = pd.read_csv(description_path, sep='\t', index_col=0)
    relations = pd.read_csv(relations_path, sep='\t', index_col=0)

Same thing for romantic movies. 

In [16]:
romance_description_path = 'Data/CoreNLP/romance_descriptions.csv'
romance_relations_path = 'Data/CoreNLP/romance_relations.csv'

if not os.path.exists(romance_description_path) and not os.path.exists(romance_relations_path):

    # Extract descriptions and relations from all romance xml files
    romance_output_dir = 'Data/CoreNLP/RomancePlotsOutputs'

    # Remove file '43849.xml' from the directory, as it is not a valid xml file
    if os.path.exists(f'{romance_output_dir}/43849.xml'):
        os.remove(f'{romance_output_dir}/43849.xml')
    
    romance_descriptions, romance_relations = extract_descriptions_relations(romance_output_dir, log_interval=1)

    # Save descriptions and relations into csv files
    romance_descriptions.to_csv(romance_description_path, sep='\t')
    romance_relations.to_csv(romance_relations_path, sep='\t')

# If we've already run the extraction, we can load the dataframe from a file
else: 
    romance_descriptions = pd.read_csv(romance_description_path, sep='\t', index_col=0)
    romance_relations = pd.read_csv(romance_relations_path, sep='\t', index_col=0)

### 4. Analyzing results

#### 4.1 Filtering relationships
Let us first get a list of everyone who is in a relationship

In [17]:
def get_characters_rom(df):
    # Get a dataframe with the unique characters from the relations dataframe and the wikipedia ID of the movie they appear in
    only_subjects = df[['subject', 'movie_id']].drop_duplicates()
    only_objects = df[['object', 'movie_id']].drop_duplicates()

    # remove rows in only_objects where the object is in the column subject of only_subjects
    only_objects = only_objects[~only_objects['object'].isin(only_subjects['subject'])]

    # Concatenate the two dataframes
    characters_df = pd.concat([only_subjects, only_objects], ignore_index=True)

    # Combine the subject and object columns into one column
    characters_df['character'] = characters_df['subject'].combine_first(characters_df['object'])
    characters_df = characters_df.drop(columns=['subject', 'object'])
    return characters_df

characters_non_romance = get_characters_rom(relations)
characters_romance = get_characters_rom(romance_relations)

print('There are {} relationships in non-romance movies, consisting of {} unique characters.'.format(len(relations), len(characters_non_romance)))
print('There are {} relationships in romance movies, consisting of {} unique characters.'.format(len(romance_relations), len(characters_romance)))

There are 7079 relationships in non-romance movies, consisting of 7060 unique characters.
There are 640 relationships in romance movies, consisting of 571 unique characters.


Now we merge the rows with the same character, and aggregate the titles

In [ ]:
# For each unique movie id and character combination, add an identifier for the character
descriptions['character_id'] = descriptions.groupby(['movie_id', 'character']).ngroup()
romance_descriptions['character_id'] = romance_descriptions.groupby(['movie_id', 'character']).ngroup()
len_desc = len(descriptions)
len_rom_desc = len(romance_descriptions)

# Combine the rows with the same character_id into one row, with the title being the concatenation of all the titles. Preserve all other columns. Ignore NaN for titles

descriptions = descriptions.groupby('character_id').agg(
    {'movie_id': 'first', 'character': 'first', 'title': lambda x: ' '.join(x.dropna()), 'agent_verbs': 'first',
     'patient_verbs': 'first', 'attributes': 'first', 'religion': 'first', 'age': 'first'})
romance_descriptions = romance_descriptions.groupby('character_id').agg(
    {'movie_id': 'first', 'character': 'first', 'title': lambda x: ' '.join(x.dropna()), 'agent_verbs': 'first',
        'patient_verbs': 'first', 'attributes': 'first', 'religion': 'first', 'age': 'first'})

# print reduction in number of rows for each dataframe
print('The number of rows in the descriptions dataframe was reduced from {} to {}.'.format(len_desc, len(descriptions)))
print('The number of rows in the romance descriptions dataframe was reduced from {} to {}.'.format(len_rom_desc, len(romance_descriptions)))


Now we merge the characters with their attributes. `full_char_rom` and `full_char` contains all characters in romantic relationships, alongside their descriptions, which also includes the gender.

In [ ]:
# Now we merge the characters with their descriptions
def map_char_attributes(char, descr):
    # Join the two dataframes based on the character name and the movie ID
    char_descr = char.merge(descr, on=['character', 'movie_id'], how='left')
    return char_descr

# convert movie_id column to int
# from movie_id column of romance_descriptions, delete all elements of string that are not digits
romance_descriptions['movie_id'] = romance_descriptions['movie_id'].str.replace(
    r'\D', '')
romance_descriptions['movie_id'] = romance_descriptions['movie_id'].astype(int)

full_char = map_char_attributes(characters_non_romance, descriptions)
full_char_rom = map_char_attributes(characters_romance, romance_descriptions)

# Merge full_char with char_df, map movie_id to Wikipedia ID and map character to Character name. Keep only column Gender from char_df, keep all columns from full_char
full_char = full_char.merge(char_df, left_on=['movie_id', 'character'], 
                            right_on=['Wikipedia ID', 'Character name'], how='left')
# From the new dataframe, drop all columns from char_df except gender
full_char = full_char.drop(['Character name', 'Wikipedia ID', 'Freebase ID', 
                            'Release date','Ethnicity', 'Date of birth', 'Height',
                            'Actor name', 'Actor age at release', 'Freebase character/map ID',
                            'Freebase character ID', 'Freebase actor ID'], axis=1)
# Same for romance movies
full_char_rom = full_char_rom.merge(char_df, left_on=['movie_id', 'character'],
                                    right_on=['Wikipedia ID', 'Character name'], how='left')
full_char_rom = full_char_rom.drop(['Character name', 'Wikipedia ID', 'Freebase ID',
                                    'Release date','Ethnicity', 'Date of birth', 'Height',
                                    'Actor name', 'Actor age at release', 'Freebase character/map ID',
                                    'Freebase character ID', 'Freebase actor ID'], axis=1)
                                    

In [ ]:
full_char = pd.read_csv('Data/CoreNLP/full_char.csv', sep='\t', index_col=0)
full_char_rom = pd.read_csv('Data/CoreNLP/full_char_rom.csv', sep='\t', index_col=0)

### Analysis titles

We first embed the titles, to then be able to cluster them

In [ ]:
#embed the titles using spacy and nltk
loading = False
if loading:
    nlp_spacy = spacy.load("en_core_web_lg")

# get the embeddings for the titles
def embed_titles(df):
    titles = df['title'].values
    embeddings = np.concatenate([nlp_spacy(title).vector.reshape(1, -1) for title in titles])
    # add the embeddings to the dataframe
    df['title_embeddings'] = list(embeddings)
    return df

title_indices = full_char[~full_char['title'].isnull() & (full_char['title'] != '')].index
char_with_title = full_char.loc[title_indices]
title_indices_rom = full_char_rom[~full_char_rom['title'].isnull() & (full_char_rom['title'] != '')].index
char_with_title_rom = full_char_rom.loc[title_indices_rom]
char_embeddings = embed_titles(char_with_title)
char_embeddings_rom = embed_titles(char_with_title_rom)

print('There are {} characters with titles in non-romance movies.'.format(len(char_embeddings)))
print('There are {} characters with titles in romance movies.'.format(len(char_embeddings_rom)))


In [ ]:
#Cluster the titles using kmeans
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Create a list of silhouette scores for different k values
silhouette_scores = []
for k in range(2, 150):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(char_embeddings['title_embeddings'].tolist())
    silhouette_scores.append(silhouette_score(char_embeddings['title_embeddings'].tolist(), kmeans.labels_))

# Plot the silhouette scores
plt.plot(range(2, 150), silhouette_scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')
plt.show()

In [ ]:
# kmeans with , only assign title to a cluster if the silhouette score is above 0.1
kmeans = KMeans(n_clusters=60, random_state=0).fit(
    char_embeddings['title_embeddings'].tolist())
char_embeddings['Cluster'] = kmeans.labels_
title_embeddings = char_embeddings[char_embeddings['Cluster'].apply(
    lambda x: silhouette_score(char_embeddings['title_embeddings'].tolist(), kmeans.labels_) > 0.05)]

In [ ]:
# import counter
from collections import Counter
# Get a dataframe, where each row is a cluster and the columns are the top 10 words in the cluster
def get_cluster_words(df, n_words=10):
    # Get the titles for each cluster
    cluster_titles = df.groupby('Cluster')['title'].apply(lambda x: ' '.join(x))
    # Get the top n words for each cluster
    cluster_words = cluster_titles.apply(lambda x: pd.Series(
        [item[0] for item in Counter(x.split()).most_common(n_words)]))
    # Add the cluster number as a column
    cluster_words['Cluster'] = cluster_words.index
    return cluster_words

# Get the top 10 words for each cluster
cluster_words = get_cluster_words(title_embeddings)
cluster_words.head(10)

#### Analysis common titles

#### Common titles men

In [ ]:
# get from char_with_title the rows with gender male
char_with_title_male = char_with_title[char_with_title['Gender'] == 'M']
char_with_title_female = char_with_title[char_with_title['Gender'] == 'F']
char_with_title_male_rom = char_with_title_rom[char_with_title_rom['Gender'] == 'M']
char_with_title_female_rom = char_with_title_rom[char_with_title_rom['Gender'] == 'F']
len_male = len(char_with_title_male)
len_male_rom = len(char_with_title_male_rom)
len_female = len(char_with_title_female)
len_female_rom = len(char_with_title_female_rom)
len_unknown = len(char_with_title) - len_male - len_female
len_unknown_rom = len(char_with_title_rom) - len_male_rom - len_female_rom

print('Known genders: \n_______________________________________________________________')
print('There are {} male characters with titles in non-romance movies.'.format(len_male))
print('There are {} female characters with titles in non-romance movies.'.format(len_female))
print('There are {} male characters with titles in romance movies.'.format(len_male_rom))
print('There are {} female characters with titles in romance movies.'.format(len_female_rom))
print('')
print('Unknown genders: \n_______________________________________________________________')
print('There are {} characters with unknown gender in non-romance movies.'.format(len_unknown))
print('There are {} characters with unknown gender in romantic movies.'.format(len_unknown_rom))

### Analysis relationships

In [ ]:
# for each relationship, add the description of both the subject and the object
def add_descriptions(relations, descriptions):
    # add the description of the subject
    relations = relations.merge(descriptions, left_on=['movie_id', 'subject'], right_on=['movie_id', 'character'], how='left')
    # add the description of the object
    relations = relations.merge(descriptions, left_on=['movie_id', 'object'], right_on=[
                  'movie_id', 'character'], how='left')
    return relations

# add the descriptions to the relationships
relations_char = add_descriptions(relations, full_char)
relations_char = relations_char.rename(columns={'subject': 'x', 'object': 'y'})

# Do the same for romance movies
relations_char_rom = add_descriptions(romance_relations, full_char_rom)
relations_char_rom = relations_char_rom.rename(columns={'subject': 'x', 'object': 'y'})


In [ ]:
# filter the relationships where title_x and title_y are both not null and not empty ("")
title_indices = relations_char[~relations_char['title_x'].isnull() & (relations_char['title_x'] != '') &
                                 ~relations_char['title_y'].isnull() & (relations_char['title_y'] != '')].index
relations_titles = relations_char.loc[title_indices][['movie_id', 'x', 'y', 'title_x', 'title_y']]

title_indices_rom = relations_char_rom[~relations_char_rom['title_x'].isnull() & (relations_char_rom['title_x'] != '') &
                                    ~relations_char_rom['title_y'].isnull() & (relations_char_rom['title_y'] != '')].index
relations_titles_rom = relations_char_rom.loc[title_indices_rom][['movie_id', 'x', 'y', 'title_x', 'title_y']]

print('There are {} relationships with titles for both persons in the couple in non-romance movies.'.format(len(relations_titles)))
print('There are {} relationships with titles for both persons in the couple in romance movies.'.format(len(relations_titles_rom)))

In [ ]:
# find rows title x and title y are the same
same_title = relations_titles[relations_titles['title_x'] == relations_titles['title_y']]
same_title

In [ ]:
# Get the top 10 pairs of title_x and title_y appearing together
relations_titles.groupby(['title_x', 'title_y']).size().sort_values(ascending=False).head(10)


### Analyzing attributes

#### Attributes for characters in relationships

Let us first look at the most common attributes for characters in relationships


In [ ]:
attributes = full_char['attributes']
attributes_rom = full_char_rom['attributes']

# Get a dictionary with the attributes as keys and the number of times they appear as values
def get_attribute_counts(attributes):
    attribute_counts = {}
    for attribute_list in attributes:
        # check if the attribute list is not NaN
        if not pd.isnull(attribute_list):
            # # remove first and last character (the brackets)
            attribute_list = attribute_list[1:-1]
            # remove all apostrophes
            attribute_list = attribute_list.replace("'", "")
            # remove all spaces
            attribute_list = attribute_list.replace(" ", "")
            # # convert string to list
            attribute_list = attribute_list.split(',')
            # iterate over the attributes in the list
            for attribute in attribute_list:
                if attribute in attribute_counts:
                    attribute_counts[attribute] += 1
                else:
                    attribute_counts[attribute] = 1
    return attribute_counts

# Get the attribute counts for non-romance movies
attribute_counts = get_attribute_counts(attributes)
# Get the attribute counts for romance movies
attribute_counts_rom = get_attribute_counts(attributes_rom)



In [ ]:
# Find the attributes that appear most often 
def get_most_common_attributes(attribute_counts, n_attributes):
    # Sort the attribute counts from highest to lowest
    sorted_attributes = sorted(attribute_counts.items(), key=lambda x: x[1], reverse=True)
    # Get the top n_attributes
    most_common_attributes = sorted_attributes[:n_attributes]
    return most_common_attributes

# Get the most common attributes for non-romance movies
most_common_attributes = get_most_common_attributes(attribute_counts, 10)
# Get the most common attributes for romance movies
most_common_attributes_rom = get_most_common_attributes(attribute_counts_rom, 10)

# Plot the 10 most common attributes for non-romance movies in a seaborn barplot, do the same for romance movies, plot side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.barplot(x=[attribute[0] for attribute in most_common_attributes], y=[attribute[1] for attribute in most_common_attributes], ax=ax1)
sns.barplot(x=[attribute[0] for attribute in most_common_attributes_rom], y=[attribute[1] for attribute in most_common_attributes_rom], ax=ax2)
# shared title
fig.suptitle('10 most common attributes in non-romance and romance movies')
# titles for the subplots
ax1.set_title('Non-romance movies')
ax2.set_title('Romance movies')
# log scale for y axis
ax1.set_yscale('log')
ax2.set_yscale('log')
# pastel palette
sns.set_palette("pastel")
# rotate x labels
plt.setp(ax1.get_xticklabels(), rotation=90)
plt.setp(ax2.get_xticklabels(), rotation=90)
# y labels
ax1.set_ylabel('Number of appearances')

plt.show()

###

In [ ]:
full_char

### OLD CODE FROM HERE ON OUT



In [ ]:
# Visualize the clusters in a 3d diagram
from sklearn.manifold import TSNE

# Reduce the dimensionality of the embeddings to 3, store each coordinate in a column
tsne = TSNE(n_components=3, random_state=0)
title_embeddings['X'] = tsne.fit_transform(title_embeddings['Embedding'].tolist())[:,0]
title_embeddings['Y'] = tsne.fit_transform(title_embeddings['Embedding'].tolist())[:,1]
title_embeddings['Z'] = tsne.fit_transform(title_embeddings['Embedding'].tolist())[:,2]

# Plot the clusters in a 3d diagram
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(title_embeddings['X'], title_embeddings['Y'], title_embeddings['Z'], c=title_embeddings['Cluster'])
plt.show()


In [ ]:
# Print 5 titles from each cluster
for i in range(24):
    print('Cluster {}:'.format(i))
    print(title_embeddings[title_embeddings['Cluster'] == i]['Title'].sample(5).values)
    print()


In [ ]:
# Plot the 10 most common character roles 
fig, ax = plt.subplots(figsize=(10, 5))

sns.countplot(x='Relation', data=title_df, order=title_df.groupby(['Relation']).count().sort_values(by = 'Wikipedia ID', ascending=False).head(10).index, ax=ax)

ax.set_title('Most common character role in romance movies')
ax.set_ylabel('Number of characters')
xlabels = ['{}'.format(x) for x in title_df.groupby(['Relation']).count().sort_values(by = 'Wikipedia ID', ascending=False).head(10).index]
ax.set_xticklabels(xlabels)
ax.set_xlabel('')
sns.set_style('darkgrid')
sns.set_palette('flare')
plt.show()